# Recursive life insurance models

In the previous section on the basic theory of life insurance modeling we calculated the **expected present value**. We wrote code that iterated through years and calculated probabilities.


## Revisiting our iterative code

Here is an iterative model we wrote in the previous notebook.

In [2]:
probability_alive = 1
mortality_rate = .01
insurance_term_limit = 10
insurance_amount = 100_000
interest_rate = .05
discount_factor = (1 + interest_rate) ** -1

death_probabilities = {}
for t in range(insurance_term_limit):
    probability_death = probability_alive * mortality_rate
    death_probabilities[t] = probability_death
    probability_alive -= probability_death

expected_present_value = sum(insurance_amount * death_probabilities[t] * pow(discount_factor, t+1) for t in death_probabilities.keys())
print(f"{expected_present_value = }")

expected_present_value = 7413.130964790028


This code works, but we want something that is easier to reason about. People generally prefer expressing formulas in a recursive style as below.

In [3]:
def deaths(t):
    """probability of dying in [t, t+1)"""
    return alive(t) * mortality_rate

def alive(t):
    """probability of being alive at time t"""
    if t <= 0:
        return 1
    return alive(t-1) - deaths(t-1)

print(f"recursive {deaths(9)=}")
print(f"iterative {death_probabilities[9]=}")
print("Same result, nice")

recursive deaths(9)=0.00913517247483641
iterative death_probabilities[9]=0.00913517247483641
Same result, nice


## The problem with recursion

Let's add some print statements and see if we can trace what the code is doing.

In [4]:
def dead(t):
    print(f"dead({t})")
    return alive(t) * mortality_rate

def alive(t):
    print(f"alive({t})")
    if t <= 0:
        return 1
    return alive(t-1) - dead(t-1)

alive(2)

alive(2)
alive(1)
alive(0)
dead(0)
alive(0)
dead(1)
alive(1)
alive(0)
dead(0)
alive(0)


0.9801

From the logs above, we can imagine that the function calls to calculate `alive(2)` are structured in the following way.

```
               alive(2)
            _______|__________
           /                  \
        alive(1)             dead(1)
       /        \              |
alive(0)       dead(0)       alive(1)
                 |          /        \
              alive(0)   alive(0)   dead(0)
                                       |
                                    alive(0)
                                       
```

The number of function calls grows exponentially. `alive(t)` will call `alive(t-1)` at least twice, causing `alive(t-2)` to be called at least 4 times, and so on until our base case `alive(0)` is hit. Let's do another experiment to verify the exponential growth.


In [5]:
def experiment(t):
    def dead(t):
        """probability of dying in [t, t+1)"""
        return alive(t) * mortality_rate
    total = 0
    def alive(t):
        """probability of being alive at time t"""
        nonlocal total
        total += 1
        if t <= 0:
            return 1
        return alive(t-1) - dead(t-1)
    alive(t)
    return total

for t in range(5):
    print(f"alive({t}) makes {experiment(t)} calls to alive")



alive(0) makes 1 calls to alive
alive(1) makes 3 calls to alive
alive(2) makes 7 calls to alive
alive(3) makes 15 calls to alive
alive(4) makes 31 calls to alive


The formula appears to be `pow(2, t+1) - 1`.

## Fixing the problem with recursion

We want to write recursive life insurance models without making an exponential number of function calls. The solution is this, if we have calculated `alive(5)` once, we store the result and just return that the next time `alive(5)` needs to be calculated. This avoids making an exponential number of recursive calls.

In [8]:
dead_cache = {}
def dead(t):
    if t in dead_cache:
        return dead_cache[t]
    print(f"dead({t})")
    dead_cache[t] = alive(t) * mortality_rate
    return dead_cache[t]

alive_cache={}
def alive(t):
    if t in alive_cache:
        return alive_cache[t]
    print(f"alive({t})")
    if t <= 0:
        alive_cache[t] = 1
    else:
        alive_cache[t] = alive(t-1) - dead(t-1)
    return alive_cache[t]

alive(2)

alive(2)
alive(1)
alive(0)
dead(0)
dead(1)


0.9801

```
                   alive(2)
                ______|______
               /             \
           alive(1)         dead(1)
           /     \           
    alive(0)     dead(0)   
```

Only the calls to functions that did not come from the cache are shown above. See that we never calculate the same value twice, this makes the diagram much smaller than before.

Another experiment but using the built-in `functools.cache` which does the same thing as our cache from before.

In [7]:
import functools

def experiment(t):
    @functools.cache
    def dead(t):
        """probability of dying in [t, t+1)"""
        return alive(t) * mortality_rate
    total = 0
    @functools.cache
    def alive(t):
        """probability of being alive at time t"""
        nonlocal total
        total += 1
        if t <= 0:
            return 1
        return alive(t-1) - dead(t-1)
    alive(t)
    return total

for t in range(5):
    print(f"alive({t}) makes {experiment(t)} calls to alive")

alive(0) makes 1 calls to alive
alive(1) makes 2 calls to alive
alive(2) makes 3 calls to alive
alive(3) makes 4 calls to alive
alive(4) makes 5 calls to alive


We no longer have an exponential number of function calls being made.

## Discussion

What we are talking about here is well known and not specific to actuarial science. Look up **memoization and recursion**. Coding interview questions on **dynamic programming** use the `functools.cache` decorator to prevent exponential blowups similar to what we observed here.

Here are some examples of interview questions that can be solved using top-down dynamic programming.

* Easy: https://leetcode.com/problems/climbing-stairs/description/
* Easy: https://leetcode.com/problems/min-cost-climbing-stairs/description/
* Medium: https://leetcode.com/problems/knight-probability-in-chessboard/description/
* Hard: https://leetcode.com/problems/sum-of-distances-in-tree/description/
* Hard: https://leetcode.com/problems/handshakes-that-dont-cross/description/